###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [41]:
import geodict
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces

# American Community Survey 2006-2010 5 Year Estimates

In [43]:
#read in API key
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [44]:
#variable containing Census API key
api_key = keys_dict_2['CENSUS']

## Read In Data Guide

In [45]:
dataguide = pd.read_csv('../../Data Guides/DATA GUIDE ACS 2010 5YR.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [46]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
dg2 = dataguide[dataguide['ID'].between(47, 92)]
dg3 = dataguide[dataguide['ID'].between(93, 138)]
dg4 = dataguide[dataguide['ID'].between(139, 184)]
dg5 = dataguide[dataguide['ID'].between(185, 230)]
dg6 = dataguide[dataguide['ID'].between(231, 276)]
dg7 = dataguide[dataguide['ID'].between(277, 322)]
dg8 = dataguide[dataguide['ID'].between(323, 368)]
dg9 = dataguide[dataguide['ID'].between(369, 414)]
dg10 = dataguide[dataguide['ID'].between(415, 460)]
dg11 = dataguide[dataguide['ID'].between(461, 506)]
dg12 = dataguide[dataguide['ID'].between(507, 552)]
dg13 = dataguide[dataguide['ID'].between(553, 598)]
dg14 = dataguide[dataguide['ID'].between(599, 644)]
dg15 = dataguide[dataguide['ID'].between(645, 690)]
dg16 = dataguide[dataguide['ID'].between(691, 736)]
dg17 = dataguide[dataguide['ID'].between(737, 782)]
dg18 = dataguide[dataguide['ID'].between(783, 828)]
dg19 = dataguide[dataguide['ID'].between(829, 874)]
dg20 = dataguide[dataguide['ID'].between(875, 920)]
dg21 = dataguide[dataguide['ID'].between(921, 966)]
dg22 = dataguide[dataguide['ID'].between(967, 1012)]
dg23 = dataguide[dataguide['ID'].between(1013, 1058)]
dg24 = dataguide[dataguide['ID'].between(1059, 1104)]
dg25 = dataguide[dataguide['ID'].between(1105, 1150)]
dg26 = dataguide[dataguide['ID'].between(1151, 1196)]
dg27 = dataguide[dataguide['ID'].between(1197, 1242)]
dg28 = dataguide[dataguide['ID'].between(1243, 1287)]
dg29 = dataguide[dataguide['ID'].between(1288, 1332)]
dg30 = dataguide[dataguide['ID'].between(1333, 1377)]
dg31 = dataguide[dataguide['ID'].between(1378, 1422)]
dg32 = dataguide[dataguide['ID'].between(1423, 1467)]
dg33 = dataguide[dataguide['ID'].between(1468, 1512)]
dg34 = dataguide[dataguide['ID'].between(1513, 1557)]
dg35 = dataguide[dataguide['ID'].between(1558, 1602)]
dg36 = dataguide[dataguide['ID'].between(1603, 1647)]
dg37 = dataguide[dataguide['ID'].between(1648, 1692)]
dg38 = dataguide[dataguide['ID'].between(1693, 1737)]
dg39 = dataguide[dataguide['ID'].between(1738, 1782)]
dg40 = dataguide[dataguide['ID'].between(1783, 1827)]
dg41 = dataguide[dataguide['ID'].between(1828, 1872)]
dg42 = dataguide[dataguide['ID'].between(1873, 1917)]
dg43 = dataguide[dataguide['ID'].between(1918, 1962)]
dg44 = dataguide[dataguide['ID'].between(1963, 2007)]
dg45 = dataguide[dataguide['ID'].between(2008, 2052)]
dg46 = dataguide[dataguide['ID'].between(2053, 2097)]
dg47 = dataguide[dataguide['ID'].between(2098, 2142)]
dg48 = dataguide[dataguide['ID'].between(2143, 2187)]
dg49 = dataguide[dataguide['ID'].between(2188, 2232)]
dg50 = dataguide[dataguide['ID'].between(2233, 2277)]
dg51 = dataguide[dataguide['ID'].between(2278, 2322)]
dg52 = dataguide[dataguide['ID'].between(2323, 2367)]
dg53 = dataguide[dataguide['ID'].between(2368, 2411)]

In [47]:
dfs = [dg1, dg2, dg3, dg4, dg5, dg6, dg7, dg8, dg9, dg10, dg11, dg12, dg13, dg14, dg15, dg16, dg17, dg18, dg19, dg20, dg21, dg22, dg23, dg24, dg25, 
       dg26, dg27, dg28, dg29, dg30, dg31, dg32, dg33, dg34, dg35, dg36, dg37, dg38, dg39, dg40, dg41, dg42, dg43, dg44, dg45, dg46, dg47, dg48, dg49,
       dg50, dg51, dg52, dg53]

In [48]:
droppers = ['ID', 'Data Point']
for df in dfs:
    df.drop(droppers, axis = 1, inplace = True)

In [49]:
#url string and list parameters for column head and tail
url_str= 'https://api.census.gov/data/2010/acs/acs5?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'

In [50]:
results = []
for df in dfs:
    dataguide = df
    for col_name, col_data in df.iteritems():
        var_list = list(dataguide['Variable']) #make variables list
        var_list = deque(var_list)
        var_list.appendleft(head2)
        var_list.appendleft(head1)
        col_list = list(dataguide['Column Name']) #make columns list
        col_list.append(tail_cols1)
        col_list.append(tail_cols2)
        col_list = deque(col_list)
        col_list.appendleft(head2)
        col_list.appendleft(head1)
        predicates= {} #call for all counties in state of TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "county:*"
        predicates["in"]= "state:47"                                                             
        data = requests.get(url_str, params= predicates)                                                                
        col_names = col_list
        df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
        predicates= {} #call for all counties in state of KY
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "county:*"
        predicates["in"]= "state:21"                                                             
        data = requests.get(url_str, params= predicates)                                                                
        col_names = col_list
        kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
        df = pd.concat([df, kycos], axis = 0)
        predicates= {} #call for all places in TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "place:*"
        predicates["in"]= "state:47"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        places=places.loc[places['GEO_ID'].isin(censusplaces)] #filter for places in the region
        df = pd.concat([df, places], axis = 0)
        predicates= {} #call for all places in KY
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "place:*"
        predicates["in"]= "state:21"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        places=places.loc[places['GEO_ID'].isin(censusplaces)] #filter for places of concern in KY
        df = pd.concat([df, places], axis = 0)
        col_list.remove(tail_cols2) #adjust the column list for different formats of geos
        predicates= {} #call for state of TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "state:47"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
        df = pd.concat([df, state], axis = 0)
        predicates= {} #call for US
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "us:*"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
        df = pd.concat([df, national], axis = 0)
        results.append(df)
new_df = pd.concat(results, axis = 1)
new_df = new_df.transpose().reset_index(drop = False).drop_duplicates()
new_df.columns = new_df.iloc[0]
new_df = new_df.set_index('NAME').transpose().drop(columns = ['StateFIPS', 'GeoFIPS']).reset_index(drop = True)
new_df = new_df.rename_axis(None, axis = 1)
print('Okay Finished')

Okay Finished


In [51]:
new_df.head()

NAME          GEO_ID hhsize_avg units_allhousing occupancy_total_series occupancy_occupiedunits occupancy_vacantunits  \
0   Cheatham County, Tennessee  0500000US47021       2.67            15425                  15425                   14198                  1227   
1   Davidson County, Tennessee  0500000US47037       2.34           280062                 280062                  252477                 27585   
2    Dickson County, Tennessee  0500000US47043       2.57            20465                  20465                   18889                  1576   
3    Houston County, Tennessee  0500000US47083       2.40             4163                   4163                    3392                   771   
4  Humphreys County, Tennessee  0500000US47085       2.43             8878                   8878                    7555                  1323   

  units_total_series units_one_detached units_one_attached units_two units_threetofour units_fivetonine units_tentonineteen units_twentytofortynine  \
0              15425              12298                207       144               217              172                 231                      82   
1             280062             150843              20840     15613              9418            16826               30358                   17467   
2              20465              14715                184       515               456              618                 424                     201   
3               4163               2904                 36        89                96               15                  24                      11   
4               8878               6342                 84       160               133              123                  11                      89   

  units_fiftyormore units_mobilehome units_boatrvvanetc hhtype_total_series hhtype_familyhh hhtype_familyhh_marriedcouplefam  \
0                39             2035                  0               14198           10659                             8312   
1             14595             4093                  9              252477          141879                            94354   
2                54             3298                  0               18889           13333                            10211   
3                 6              982                  0                3392            2336                             1679   
4                 8             1897                 31                7555            5192                             4082   

  hhtype_familyhh_otherfam hhtype_familyhh_malenospouse hhtype_familyhh_femalenospouse hhtype_nonfamhh hhtype_nonfamhh_householderalone  \
0                     2347                          874                           1473            3539                             2940   
1                    47525                        10697                          36828          110598                            89509   
2                     3122                          788                           2334            5556                             4673   
3                      657                          295                            362            1056                              892   
4                     1110                          262                            848            2363                             1901   

  hhtype_nonfamhh_householdernotalone hhincome_median hhincome_total_series hhincome_lessthan10000 hhincome_10to14999 hhincome_15to19999  \
0                                 599           52585                 14198                    596                709                715   
1                               21089           45668                252477                  21983              14802              13855   
2                                 883           44554                 18889                   1406               1362               1090   
3                                 164           33738                  3

In [52]:
data = new_df

In [53]:
data = data.set_index(['NAME', 'GEO_ID']).transpose()
data.head()

NAME,"Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Macon County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee","Allen County, Kentucky","Simpson County, Kentucky","Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Cedar Hill city, Tennessee","Charlotte town, Tennessee","Clarksville city, Tennessee","Columbia city, Tennessee","Coopertown town, Tennessee","Cross Plains city, Tennessee","Cumberland City town, Tennessee","Dickson city, Tennessee","Dover city, Tennessee","Eagleville city, Tennessee","Erin city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee","Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Hendersonville city, Tennessee","Kingston Springs town, Tennessee","Lafayette city, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee","McEwen city, Tennessee","Millersville city, Tennessee","Mitchellville city, Tennessee","Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee","New Johnsonville city, Tennessee","Nolensville town, Tennessee","Oak Hill city, Tennessee","Pegram town, Tennessee","Pleasant View city, Tennessee","Portland city, Tennessee","Ridgetop city, Tennessee","Slayden town, Tennessee","Smyrna town, Tennessee","Springfield city, Tennessee","Spring Hill city, Tennessee","Tennessee Ridge town, Tennessee","Thompson's Station town, Tennessee","Vanleer town, Tennessee","Watertown city, Tennessee","Waverly city, Tennessee","Westmoreland town, Tennessee","White Bluff town, Tennessee","White House city, Tennessee","Franklin city, Kentucky","Scottsville city, Kentucky",Tennessee,United States
GEO_ID,0500000US47021,0500000US47037,0500000US47043,0500000US47083,0500000US47085,0500000US47111,0500000US47119,0500000US47125,0500000US47147,0500000US47149,0500000US47161,0500000US47165,0500000US47169,0500000US47187,0500000US47189,0500000US21003,0500000US21213,1600000US4700200,1600000US4702180,1600000US4704620,1600000US4705140,1600000US4708280,1600000US4709880,1600000US4711980,1600000US4713080,1600000US4715160,1600000US4716540,1600000US4716980,1600000US4718420,1600000US4718820,1600000US4720620,1600000US4721400,1600000US4722360,1600000US4724320,1600000US4725440,1600000US4727020,1600000US4727740,1600000US4728540,1600000US4729920,1600000US4730960,1600000US4733280,1600000US4739660,1600000US4740160,1600000US4741200,1600000US4741520,1600000US4744840,1600000US4748980,1600000US4749460,1600000US4750780,1600000US4751080,1600000US4751560,1600000US4752006,1600000US4752820,1600000US4753460,1600000US4754780,1600000US4757480,1600000US4759560,1600000US4760280,1600000US4763140,1600000US4769080,1600000US4769420,1600000US4770500,1600000US4770580,1600000US4773460,1600000US4773900,1600000US4776860,1600000US4778320,1600000US4778560,1600000US4779420,1600000US4779980,1600000US4780200,1600000US2128918,1600000US2169114,0400000US47,0100000US
hhsize_avg,2.67,2.34,2.57,2.40,2.43,2.61,2.45,2.64,2.69,2.65,2.44,2.60,2.75,2.78,2.63,2.47,2.52,2.47,2.38,2.71,1.47,2.99,2.43,2.98,2.57,2.63,2.32,2.91,2.88,2.78,2.40,2.26,2.00,2.25,2.75,2.87,2.57,2.39,2.54,2.67,2.59,2.64,2.27,2.99,2.45,2.31,2.56,2.40,2.80,2.31,2.49,2.33,2.38,3.09,2.44,2.77,2.80,2.66,2.76,2.80,2.55,2.64,2.94,2.42,2.74,2.48,2.75,2.18,2.57,2.64,2.68,2.45,2.22,2.49,2.59
units_allhousing,15425,280062,20465,4163,8878,9743,34641,67511,25607,99507,6675,64071,3335,65903,43960,9136,7456,238,1647,1164,380,12155,747,136,418,52715,15888,1462,590,186,6258,816,279,757,2822,1756,24366

In [54]:
numcols = list(data.columns)
numcols
data[numcols] = data[numcols].astype(float)

In [55]:
GNRCCounties = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                data[('Rutherford County, Tennessee', '0500000US47149')]]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                   data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                   data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                   data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                   data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                   data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                   data[('Rutherford County, Tennessee', '0500000US47149')],data[('Maury County, Tennessee', '0500000US47119')]]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
               data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
               data[('Williamson County, Tennessee', '0500000US47187')],data[('Rutherford County, Tennessee', '0500000US47149')],
               data[('Maury County, Tennessee', '0500000US47119')]]
data['MPO'] = sum(MPOCounties)
RuthInc = [data[('Eagleville city, Tennessee', '1600000US4722360')],data[('La Vergne city, Tennessee', '1600000US4741200')],
           data[('Murfreesboro city, Tennessee', '1600000US4751560')],data[('Smyrna town, Tennessee', '1600000US4769420')]]
data[('Rutherford Incorporated', 'None')] = sum(RuthInc)
data[('Rutherford Unincorporated', 'None')] = data[('Rutherford County, Tennessee', '0500000US47149')] - data[('Rutherford Incorporated', 'None')]
WilsonInc = [data[('Lebanon city, Tennessee', '1600000US4741520')],data[('Mount Juliet city, Tennessee', '1600000US4750780')],
             data[('Watertown city, Tennessee', '1600000US4778320')]]
data[('Wilson Incorporated', 'None')] = sum(WilsonInc)
data[('Wilson Unincorporated', 'None')] = data[('Wilson County, Tennessee', '0500000US47189')] - data[('Wilson Incorporated', 'None')]
CheathInc = [data[('Ashland City town, Tennessee', '1600000US4702180')],data[('Kingston Springs town, Tennessee', '1600000US4739660')],
             data[('Pegram town, Tennessee', '1600000US4757480')],data[('Pleasant View city, Tennessee', '1600000US4759560')]]
data[('Cheatham Incorporated', 'None')] = sum(CheathInc)
data[('Cheatham Unincorporated', 'None')] = data[('Cheatham County, Tennessee', '0500000US47021')] - data[('Cheatham Incorporated', 'None')]
DicksInc = [data[('Burns town, Tennessee', '1600000US4709880')],data[('Charlotte town, Tennessee', '1600000US4713080')],
            data[('Dickson city, Tennessee', '1600000US4720620')],data[('Slayden town, Tennessee', '1600000US4769080')],
            data[('Vanleer town, Tennessee', '1600000US4776860')],data[('White Bluff town, Tennessee', '1600000US4779980')]]
data[('Dickson Incorporated', 'None')] = sum(DicksInc)
data[('Dickson Unincorporated', 'None')] = data[('Dickson County, Tennessee', '0500000US47043')] - data[('Dickson Incorporated', 'None')]
HumphInc = [data[('McEwen city, Tennessee', '1600000US4744840')],data[('New Johnsonville city, Tennessee', '1600000US4752820')],
            data[('Waverly city, Tennessee', '1600000US4778560')]]
data[('Humphreys Incorporated', 'None')] = sum(HumphInc)
data[('Humphreys Unincorporated', 'None')] = data[('Humphreys County, Tennessee', '0500000US47085')] - data[('Humphreys Incorporated', 'None')]
data[('Montgomery Incorporated', 'None')] = data[('Clarksville city, Tennessee', '1600000US4715160')]
data[('Montgomery Unincorporated', 'None')] = data[('Montgomery County, Tennessee', '0500000US47125')] - data[('Montgomery Incorporated', 'None')]

In [56]:
data = data.transpose().reset_index()
data.head()

NAME          GEO_ID  hhsize_avg  units_allhousing  occupancy_total_series  occupancy_occupiedunits  \
0   Cheatham County, Tennessee  0500000US47021        2.67           15425.0                 15425.0                  14198.0   
1   Davidson County, Tennessee  0500000US47037        2.34          280062.0                280062.0                 252477.0   
2    Dickson County, Tennessee  0500000US47043        2.57           20465.0                 20465.0                  18889.0   
3    Houston County, Tennessee  0500000US47083        2.40            4163.0                  4163.0                   3392.0   
4  Humphreys County, Tennessee  0500000US47085        2.43            8878.0                  8878.0                   7555.0   

   occupancy_vacantunits  units_total_series  units_one_detached  units_one_attached  units_two  units_threetofour  units_fivetonine  \
0                 1227.0             15425.0             12298.0               207.0      144.0              217.0             172.0   
1                27585.0            280062.0            150843.0             20840.0    15613.0             9418.0           16826.0   
2                 1576.0             20465.0             14715.0               184.0      515.0              456.0             618.0   
3                  771.0              4163.0              2904.0                36.0       89.0               96.0              15.0   
4                 1323.0              8878.0              6342.0                84.0      160.0              133.0             123.0   

   units_tentonineteen  units_twentytofortynine  units_fiftyormore  units_mobilehome  units_boatrvvanetc  hhtype_total_series  hhtype_familyhh  \
0                231.0                     82.0               39.0            2035.0                 0.0              14198.0          10659.0   
1              30358.0                  17467.0            14595.0            4093.0                 9.0             252477.0         141879.0   
2                424.0                    201.0               54.0            3298.0                 0.0              18889.0          13333.0   
3                 24.0                     11.0                6.0             982.0                 0.0               3392.0           2336.0   
4                 11.0                     89.0                8.0            1897.0                31.0               7555.0           5192.0   

   hhtype_familyhh_marriedcouplefam  hhtype_familyhh_otherfam  hhtype_familyhh_malenospouse  hhtype_familyhh_femalenospouse  hhtype_nonfamhh  \
0                            8312.0                    2347.0                         874.0                          1473.0           3539.0   
1                           94354.0                   47525.0                       10697.0                         36828.0         110598.0   
2                           10211.0                    3122.0                         788.0                          2334.0           5556.0   
3                            1679.0                     657.0                         295.0                           362.0           1056.0   
4                            4082.0                    1110.0                         262.0                           848.0           2363.0   

   hhtype_nonfamhh_householderalone  hhtype_nonfamhh_householdernotalone  hhincome_median  hhincome_total_series  hhincome_lessthan10000  \
0                            2940.0                                599.0          52585.0                14198.0                   596.0   
1                           89509.0                              21089.0          45668.0               252477.0                 21983.0   
2                            4673.0                                883.0          44554.0                18889.0                  1406.0   
3                             892.0                                164.0          33738.0                 3392.0                   3

In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Columns: 2380 entries, NAME to earn_fulltime_ median_test2
dtypes: float64(2378), object(2)
memory usage: 1.6+ MB


In [59]:
data.to_feather('../../Raw Data/ACS20105YR')